In [9]:
# Check redshift/mass bins
from mceh import utility as ut # v3.0.0-alpha.2
import numpy as np
import matplotlib.pyplot as plt
import mceh.utility as ut
import mceh.find_luminosity as fl
import astropy.units as u

In [2]:
efeds = ut.init('efeds')

In [3]:
z = efeds['Z_BEST_COMB']
mass = efeds['median_500c_lcdm'].value

In [4]:
print(min(z), max(z))

0.017000000923871994 1.3038


In [5]:
def fit_band(z):
    if 0.0 < z and z < 0.35:
        return 'r'
    if 0.35 <= z and z < 0.75:
        return 'i'
    if 0.75 <= z and z < 1.12:
        return 'z'
    else:
        return 'y'

In [6]:
efeds = efeds[efeds['low_cont_flag']]
efeds = efeds[efeds['unmasked_fraction'] > 0.6]

In [7]:
z = efeds['Z_BEST_COMB']
mass = efeds['median_500c_lcdm'].value

In [8]:
# 0, 0.35, 0.75, 1.12, 1.4
bins1 = [0, 0.35, 0.75, 1.12, 1.4]
for i in range(len(bins1) - 1):
    print(np.sum((z > bins1[i]) & (z < bins1[i + 1])))

158
136
22
1


In [9]:
def between_num(v1, v2, data):
    return np.sum((data > v1) & (data < v2))
def between_data(v1, v2, data):
    return data[(data > v1) & (data < v2)]

In [10]:
data = between_data(0, 0.35, z)

In [11]:
np.percentile(data, [0, 100/3, 200/3, 100])

0.10123
0.22553
0.29664833333333335
0.349955


In [12]:
z_ = np.linspace(0, 0.35, 4)
z_

array([0.        , 0.11666667, 0.23333333, 0.35      ])

In [13]:
i = 1
between_num(z_[i], z_[i+1], data)

51

In [14]:
z_ = [0.1, 0.23, 0.3, 0.35]
for i in range(len(z_)-1):
    print(between_num(z_[i], z_[i+1], data))

55
53
50


In [15]:
# [0.10, 0.23, 0.30, 0.35]
# 0.35, 0.75 bins1
between_num(0.35, 0.75, z) # 3

136

In [16]:
z_ = between_data(0.35, 0.75, z)
z_ = [np.round(i, 2) for i in np.percentile(z_,(100/3, 200/3))]
z_

[0.41, 0.5]

In [17]:
data

0.24744
0.197875
0.349895
0.152525
0.330435
0.3393
0.11006
0.278562
0.198095
0.26756
0.22418


In [18]:
def between_num(bins, data):
    returnme = np.array([])
    for i in range(len(bins) - 1):
        returnme = np.append(returnme, np.sum((data >= bins[i]) & (data < bins[i + 1])))
    return returnme

In [19]:
between_num([0.35, 0.41, 0.5, 0.75], z)

array([44., 48., 44.])

In [20]:
# [0,10, 0.23, 0.30, 0.35, 0.41, 0.50, 0.75]
zbin2 = np.array([0.1, 0.35, 0.75, 1.4])
def find_bin(z, z_init_bound, lowest_num):
    result_z = []
    result_m = []
    for i in range(len(z_init_bound) - 1):
        low_z, high_z = z_init_bound[i], z_init_bound[i + 1]
        this_z = between_data(low_z, high_z, z)
        num = between_num([low_z, high_z], z)
        bin_num = int(num / lowest_num) # if 120 -> 3, if 121 -> 3
        if bin_num == 0:
            bin_num = 1
        percent = np.linspace(0, 100, bin_num + 1)
        this_bin = np.round(np.percentile(this_z, percent), 2)
        this_bin[0], this_bin[-1] = low_z, high_z
        print([between_num([this_bin[j], this_bin[j + 1]], this_z) for j in range(len(this_bin) - 1)])
        if i != (len(z_init_bound) - 2):
            this_bin = np.delete(this_bin, -1)
        result_z = np.append(result_z, this_bin)
    return result_z

In [21]:
result_z = find_bin(z, zbin2, 40)
print(result_z)

[array([55.]), array([53.]), array([50.])]
[array([44.]), array([48.]), array([44.])]
[array([23.])]
[0.1  0.23 0.3  0.35 0.41 0.5  0.75 1.4 ]


/tmp/ipykernel_2534387/2508359493.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  bin_num = int(num / lowest_num) # if 120 -> 3, if 121 -> 3


In [27]:
result_m = []
cnum = []
for i in range(len(result_z) - 1):
    low_z = result_z[i]
    high_z = result_z[i + 1]
    this_m = mass[np.where((z >= low_z) & (z < high_z))]
    print('min_m, max_m =', min(this_m), max(this_m))
    low_m, high_m = (np.floor(min(this_m) * 100) / 100, 
                     np.ceil(max(this_m) * 100) / 100)
    print('low_m, high_m =', low_m, high_m)
    this_bin = find_bin(this_m, [low_m, high_m], 20)
    result_m.append(this_bin)
    cnum.append(between_num(this_bin, this_m))
print(result_m)

min_m, max_m = 13.377914404857988 14.505669974966601
low_m, high_m = 13.37 14.51
[array([28.]), array([27.])]
min_m, max_m = 13.616971107055477 14.470737195015243
low_m, high_m = 13.61 14.48
[array([27.]), array([26.])]
min_m, max_m = 13.64881597684473 14.53955930193813
low_m, high_m = 13.64 14.54
[array([25.]), array([25.])]
min_m, max_m = 13.739353848858148 14.634275929919987
low_m, high_m = 13.73 14.64
[array([22.]), array([22.])]
min_m, max_m = 13.786010397486109 14.575956489015905
low_m, high_m = 13.78 14.58
[array([26.]), array([22.])]
min_m, max_m = 13.911078490675923 14.686369365819369
low_m, high_m = 13.91 14.69
[array([23.]), array([21.])]
min_m, max_m = 14.01998024038144 14.515356824064789
low_m, high_m = 14.01 14.52
[array([23.])]
[array([13.37, 13.73, 14.51]), array([13.61, 13.87, 14.48]), array([13.64, 13.99, 14.54]), array([13.73, 14.02, 14.64]), array([13.78, 14.06, 14.58]), array([13.91, 14.17, 14.69]), array([14.01, 14.52])]


/tmp/ipykernel_2534387/2508359493.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  bin_num = int(num / lowest_num) # if 120 -> 3, if 121 -> 3


In [26]:
for i in range(len(result_z) - 1):
    low_z = result_z[i]
    high_z = result_z[i + 1]
    this_m = mass[np.where((z >= low_z) & (z < high_z))]
    print(between_num([13, 14, 15], this_m))

[49.  6.]
[38. 15.]
[25. 25.]
[19. 25.]
[18. 30.]
[ 6. 38.]
[ 0. 23.]


In [34]:
cnum = [list(i) for i in cnum]
for i in range(len(cnum)):
    for j in range(len(cnum[i])):
        cnum[i][j] = int(cnum[i][j])
print(cnum)

[[28, 27], [27, 26], [25, 25], [22, 22], [26, 22], [23, 21], [23]]


In [36]:
result = {}
result['zbins'] = result_z
result['mbins'] = result_m
result['cnum'] = cnum
ut.pickle_dump(result, 'result/zmbins20241108.pickle')

In [37]:
result = ut.pickle_load('result/zmbins20241108.pickle')
result['cnum']

[[28, 27], [27, 26], [25, 25], [22, 22], [26, 22], [23, 21], [23]]

In [10]:
efeds, rd = ut.init('efeds', 'rd')

/home/hylin/.conda/envs/env1/lib/python3.9/site-packages/astropy/units/core.py:2126: UnitsWarning: 'solMass / littleh' did not parse as fits unit: At col 10, Unit 'littleh' not supported by the FITS standard.  If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see https://docs.astropy.org/en/latest/units/combining_and_defining.html
  warnings.warn(msg, UnitsWarning)
/home/hylin/.conda/envs/env1/lib/python3.9/site-packages/astropy/units/core.py:2126: UnitsWarning: 'solMass / littleh' did not parse as unit: At col 10, littleh is not a valid unit.  If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see https://docs.astropy.org/en/latest/units/combining_and_defining.html
  warnings.warn(msg, UnitsWarning)


In [11]:
efeds = efeds[efeds['low_cont_flag'] & (efeds['unmasked_fraction'] > 0.6)]
max_r = max(efeds['r500c_lcdm'])
max_r

<Quantity 6.2991881 arcmin>

In [6]:
rd['RA'].unit

Unit("deg")

In [12]:
original_efeds = ut.init('efeds')

/home/hylin/.conda/envs/env1/lib/python3.9/site-packages/astropy/units/core.py:2126: UnitsWarning: 'solMass / littleh' did not parse as fits unit: At col 10, Unit 'littleh' not supported by the FITS standard.  If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see https://docs.astropy.org/en/latest/units/combining_and_defining.html
  warnings.warn(msg, UnitsWarning)
/home/hylin/.conda/envs/env1/lib/python3.9/site-packages/astropy/units/core.py:2126: UnitsWarning: 'solMass / littleh' did not parse as unit: At col 10, littleh is not a valid unit.  If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see https://docs.astropy.org/en/latest/units/combining_and_defining.html
  warnings.warn(msg, UnitsWarning)


In [14]:
fl.print = ut.printt
rd_p = fl.nonoverlapping_random_point(3000, max_r, 
                                      min(rd['RA'].value) * u.deg,
                                      max(rd['RA'].value) * u.deg,
                                      min(rd['DEC'].value) * u.deg,
                                      max(rd['DEC'].value) * u.deg,
                                      original_efeds['RA_CORR'],
                                      original_efeds['DEC_CORR'],
                                      original_efeds['r500c_lcdm'],
                                      rd_data=rd,
                                      coverage=60,
                                      progress=True)

[11/08 17:00:00] Finish 1/3000
[11/08 17:00:00] Finish 2/3000
[11/08 17:00:00] Finish 3/3000
[11/08 17:00:01] Finish 4/3000
[11/08 17:00:01] Finish 5/3000
[11/08 17:00:01] Finish 6/3000
[11/08 17:00:02] Finish 7/3000
[11/08 17:00:02] Finish 8/3000
[11/08 17:00:02] Finish 9/3000
[11/08 17:00:02] Finish 10/3000
[11/08 17:00:02] Finish 11/3000
[11/08 17:00:03] Finish 12/3000
[11/08 17:00:03] Finish 13/3000
[11/08 17:00:03] Finish 14/3000
[11/08 17:00:04] Finish 15/3000
[11/08 17:00:04] Finish 16/3000
[11/08 17:00:04] Finish 17/3000
[11/08 17:00:04] Finish 18/3000
[11/08 17:00:05] Finish 19/3000
[11/08 17:00:05] Finish 20/3000
[11/08 17:00:05] Finish 21/3000
[11/08 17:00:05] Finish 22/3000
[11/08 17:00:05] Finish 23/3000
[11/08 17:00:06] Finish 24/3000
[11/08 17:00:06] Finish 25/3000
[11/08 17:00:06] Finish 26/3000
[11/08 17:00:06] Finish 27/3000
[11/08 17:00:06] Finish 28/3000
[11/08 17:00:06] Finish 29/3000
[11/08 17:00:07] Finish 30/3000
[11/08 17:00:07] Finish 31/3000
[11/08 17:00:07] 

KeyboardInterrupt: 